In [3]:
# imports
import tensorflow as tf
import tensorflow.keras as keras
from keras.backend.tensorflow_backend import set_session
import numpy as np
import datetime
from contextlib import redirect_stdout
import pickle

Using TensorFlow backend.


In [4]:
# Try to load an untrained model
loaded_model = keras.models.load_model('./untrained_models_h5/20210425_05-20-56_lstm.h5')

In [11]:
# Config growth --
# https://kobkrit.com/using-allow-growth-memory-option-in-tensorflow-and-keras-dc8c8081bc96
config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
config.log_device_placement = True
sess = tf.Session(config=config)

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [12]:
# Log time for tracking
now = datetime.datetime.now().strftime('%Y%m%d_%H-%M-%S')

# Hyperparameters
lstm_hidden_units = 256
dense_hidden_units = 256
batch_size = 128
epochs = 256

# The model
model = keras.models.Sequential(name=f'{now}_lstm')

# Input layer
model.add(
    keras.layers.LSTM(
        lstm_hidden_units,
        input_shape=(100, 98),
        recurrent_dropout=0.3,
        return_sequences=True,
        name='input_lstm'
    ))

# LSTM hidden 0
model.add(keras.layers.LSTM(lstm_hidden_units,
            return_sequences=True, recurrent_dropout=0.3))

# LSTM hidden 1
model.add(keras.layers.LSTM(lstm_hidden_units))

# Batch norm
model.add(keras.layers.BatchNormalization())

# Dropout
model.add(keras.layers.Dropout(0.3))

# Dense
model.add(keras.layers.Dense(dense_hidden_units,))
model.add(keras.layers.Activation('relu'))

# Batch norm
model.add(keras.layers.BatchNormalization())

# Dropout
model.add(keras.layers.Dropout(0.3,))

# Output a single note represented by the vocab
# of the multilabeled one hot vector
model.add(keras.layers.Dense(
    98, activation='sigmoid', name='dense_output'))

model.add(keras.layers.Reshape(target_shape=(1, 98), name='reshaped_output'))

# Compile it
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

# Display it and write to file
keras.utils.plot_model(
    model, to_file=f'./inspect_figures/{now}_lstm.png', expand_nested=True, show_shapes=True)
with open(f'./inspect_figures/{now}_lstm_modelsummary.txt', 'w') as f:
    with redirect_stdout(f):
        model.summary()

In [13]:
# Make dummy input and output tensors
dummy_input =  np.random.randint(0, high=1, size=(2000, 100, 98))
dummy_output = np.random.randint(0, high=1, size=(2000, 1, 98))

In [14]:
# Attempt to train
history = model.fit(dummy_input, dummy_output, batch_size=256, epochs=6, verbose=1, validation_split=0.2)

Train on 1600 samples, validate on 400 samples
Epoch 1/6
1600/1600 [==============================] - 11s 7ms/sample - loss: 0.6782 - val_loss: 0.6442
Epoch 2/6
1600/1600 [==============================] - 3s 2ms/sample - loss: 0.6189 - val_loss: 0.5752
Epoch 3/6
1600/1600 [==============================] - 3s 2ms/sample - loss: 0.5480 - val_loss: 0.5022
Epoch 4/6
1600/1600 [==============================] - 3s 2ms/sample - loss: 0.4748 - val_loss: 0.4297
Epoch 5/6
1600/1600 [==============================] - 3s 2ms/sample - loss: 0.4031 - val_loss: 0.3608
Epoch 6/6
1600/1600 [==============================] - 3s 2ms/sample - loss: 0.3371 - val_loss: 0.2977


In [18]:
# Test pickling the history
with open(f'./history/{now}_test_pickled_history', 'wb') as fobj:
    pickle.dump(history.history, fobj)

In [19]:
# Check loading 
with open(f'./history/{now}_test_pickled_history', 'rb') as fobj:
    reloaded_history = pickle.load(fobj)

In [22]:
reloaded_history.keys()

dict_keys(['loss', 'val_loss'])